In [1]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
# you can find the latest package @ https://github.com/kubeflow/pipelines/releases
#KFP_PACKAGE = 'https://storage.googleapis.com/ml-pipeline/release/0.1.20/kfp.tar.gz'
#!pip3 install $KFP_PACKAGE --upgrade

# Starting from existing code to build on KubeFlow Pipelines 

This is a series of notebooks in which we will demonstrate how to start from an existing code base and create a pipelin in a step by step fasion. In this sample we will:

* **Create a single step pipeline from existing code**
* Break the single step to multiple pipeline steps using the sample container
* Create a separate container for each step

Reference documentation: 
* https://www.kubeflow.org/docs/pipelines/sdk/build-component/
* https://www.kubeflow.org/docs/pipelines/sdk/sdk-overview/

This sample is based on the TF sample for more details please see
* https://github.com/tensorflow/docs/blob/master/site/en/tutorials/_index.ipynb 

## Setup

In [3]:
# Set your output and project. !!!Must Do before you can proceed!!!
EXPERIMENT_NAME = 'Hellow world!'
PROJECT_NAME =  'chavoshi-dev-2'                      #'Your-Gcp-Project-Name'
OUTPUT_DIR = 'gs://chavoshi-dev-mlpipeline'          # A path for asset outputs
BASE_IMAGE='tensorflow/tensorflow:1.11.0-py3'         # Based image used in various steps of the pipeline
TARGET_IMAGE='gcr.io/%s/byoc:latest' % PROJECT_NAME # Target image that will include our final code

## Create an Experiment in the Pipeline System

Pipeline system requires an "Experiment" to group pipeline runs. You can create a new experiment, or call client.list_experiments() to get existing ones.

In [4]:
#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

# Build the pipeline

In [5]:
# Note that this notebook should be running in JupyterHub in the same cluster as the pipeline system.
# Otherwise it will fail to talk to the pipeline system.
import kfp
import kfp.dsl as dsl
from kfp.gcp import use_gcp_secret
from kubernetes import client as k8s_client
from kfp import compiler
from kfp import notebook
from kfp import components as comp

In [6]:
@dsl.python_component(
    name='mnist_e2e_single_component',
    description='Runs the enire mnist code in one single component',
    base_image=BASE_IMAGE  # note you can define the base image here, or during build time. 
)

def mnist_example_e2e():
    # Your code goes here
    # Note that you can instead use git clone or gsutil copy the code into the base image.
    
    import tensorflow as tf
    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=5)
    model.evaluate(x_test, y_test)
    
    return 


### Build a Component With the Above Function

In [7]:
from kfp import compiler

# The return value "DeployerOp" represents a step that can be used directly in a pipeline function
mnist_e2e_op = compiler.build_python_component(
    component_func=mnist_example_e2e,
    staging_gcs_path=OUTPUT_DIR,
    dependency=[kfp.compiler.VersionedDependency(name='google-api-python-client', version='1.7.0')],
    base_image=BASE_IMAGE,
    target_image=TARGET_IMAGE)

2019-07-11 17:16:47:INFO:Build an image that is based on tensorflow/tensorflow:1.11.0-py3 and push the image to gcr.io/chavoshi-dev-2/byoc:latest
2019-07-11 17:16:47:INFO:Checking path: gs://chavoshi-dev-mlpipeline...
2019-07-11 17:16:47:INFO:Generate entrypoint and serialization codes.
2019-07-11 17:16:47:INFO:Generate build files.
2019-07-11 17:16:47:INFO:Start a kaniko job for build.
2019-07-11 17:16:47:INFO:Cannot Find local kubernetes config. Trying in-cluster config.
2019-07-11 17:16:47:INFO:Initialized with in-cluster config.
2019-07-11 17:16:52:INFO:5 seconds: waiting for job to complete
2019-07-11 17:16:57:INFO:10 seconds: waiting for job to complete
2019-07-11 17:17:02:INFO:15 seconds: waiting for job to complete
2019-07-11 17:17:07:INFO:20 seconds: waiting for job to complete
2019-07-11 17:17:12:INFO:25 seconds: waiting for job to complete
2019-07-11 17:17:17:INFO:30 seconds: waiting for job to complete
2019-07-11 17:17:22:INFO:35 seconds: waiting for job to complete
2019-07

### Build a pipeline using this component

In [8]:
import kfp.dsl as dsl
@dsl.pipeline(
   name='MNIST pipeline',
   description='A sample pipeline that runs MNIST.'
)
def mnist_pipeline(
):
    mnist_e2e_task = mnist_e2e_op() 

### Complie the pipeline

In [9]:
pipeline_func = mnist_pipeline
pipeline_filename = pipeline_func.__name__ + '.pipeline.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

#### Submit the pipeline for execution

In [10]:
#Specify pipeline argument values
arguments = {}

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

#This link leads to the run information page. 
#Note: There is a bug in JupyterLab that modifies the URL and makes the link stop working